In [1]:
import requests
from bs4 import BeautifulSoup

# URLからHTMLを取得し、BeautifulSoupで解析
url = 'https://www.asahi.com/news/daily/0101.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

/Users/shuma/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import requests
from bs4 import BeautifulSoup

# 1ページ目から7ページ目までスクレイピング
for page in range(101, 108):
    url = f'https://www.asahi.com/news/daily/0{page}.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # スクレイピングのロジックをここに追加


In [ ]:
import requests
from bs4 import BeautifulSoup

# ベースURL
base_url = 'https://www.asahi.com/news/daily/'

# 文字コードをUTF-8に設定
encoding = 'utf-8'

# 1ページ目から7ページ目までスクレイピング
for page in range(101, 108):  # 0101から0107まで
    url = f'{base_url}0{page}.html'
    
    response = requests.get(url)
    response.raise_for_status()
    response.encoding = encoding

    soup = BeautifulSoup(response.text, 'html.parser')
    news_list = soup.find('ul', class_='List')

    # 最初のニュース項目を取得
    first_news_item = news_list.find('li', recursive=False)
    title = first_news_item.get_text().strip()
    link = first_news_item.find('a')['href']

    # 相対リンクを絶対リンクに変換
    if link.startswith('/'):
        link = 'https://www.asahi.com' + link

    print(f'1月{page - 100}日, {title},  {link}\n')


In [3]:
import sqlite3
import requests
from bs4 import BeautifulSoup

# データベースに接続（存在しない場合は新規作成）
conn = sqlite3.connect('news.db')
c = conn.cursor()

# ニュースデータを格納するためのテーブルを作成
c.execute('''
    CREATE TABLE IF NOT EXISTS news (
        date TEXT,
        title TEXT,
        link TEXT
    )
''')

# ベースURL
base_url = 'https://www.asahi.com/news/daily/'

# 文字コードをUTF-8に設定
encoding = 'utf-8'

# 1ページ目から7ページ目までスクレイピング
for page in range(101, 108):  # 0101から0107まで
    url = f'{base_url}0{page}.html'
    
    response = requests.get(url)
    response.raise_for_status()
    response.encoding = encoding

    soup = BeautifulSoup(response.text, 'html.parser')
    news_list = soup.find('ul', class_='List')

    # 最初のニュース項目を取得
    first_news_item = news_list.find('li', recursive=False)
    title = first_news_item.get_text().strip()
    link = first_news_item.find('a')['href']

    # 相対リンクを絶対リンクに変換
    if link.startswith('/'):
        link = 'https://www.asahi.com' + link

    # データベースにデータを挿入
    c.execute('INSERT INTO news (date, title, link) VALUES (?, ?, ?)', 
              (f'1月{page - 100}日', title, link))

    # 挿入したデータをコミット
    conn.commit()

# データベースのクローズ
conn.close()
